In [1]:
import sys
sys.path.append('../')

from models import utils, Simple_CNN, Simple_LSTM, Hierarchical

import torch
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import matplotlib.pyplot as plt

import numpy as np

import yaml

import time

from tqdm.notebook import tqdm

In [2]:
with open("./yamls/HierarchyCNN.yaml", "r") as f:
  net_params = yaml.load(f, Loader=yaml.FullLoader)

net_organization = net_params["net_organization"]
node_params = net_params["node_params"]
net_architecture = net_params["net_architecture"]

random_seed = 42

net_name = "test"

transform_img = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    ]
)

transform_labels = transforms.Compose(
    [transforms.ToTensor()]
)

dataset = utils.HierarchicalDataset("../datasets/full_dataset_df.csv",
                                    "../spectrograms/full_dataset",
                                    "../mfcc/",
                                    "gbifID",
                                    ["phylum", "class", "family"],
                                    transform=transform_img, preload=False)

H = Hierarchical.HierarchicalClassification(net_architecture, node_params, net_organization, dataset, load=True)

Loaded from ./networks/hierarchical/family_Arachnida.pth:
SimpleCNN(
  (conv): ModuleList(
    (0): Conv2d(3, 8, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (1): Conv2d(8, 16, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (3): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (4): Conv2d(64, 64, kernel_size=(2, 2), stride=(1, 1), padding=same)
    (5): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1), padding=same)
  )
  (bnorm): ModuleList(
    (0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [3]:
df = H.dataset.csv
df.head()

,gbifID,identifier,species,genus,family,class,phylum,file_name
0,1572324720,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
1,1572324719,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
2,1572324718,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
3,1572324717,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...
4,1572324716,http://www.tierstimmenarchiv.de/recordings/Cre...,Crex crex,Crex,Rallidae,Aves,Chordata,Chordata/Aves/Rallidae/Crex/Crex_crex/15723247...


In [4]:
label_dict = {}

arach_index = df[df["class"]=="Arachnida"].index
arach = []

for index in arach_index:
  arach.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Arachnida"] = list(set(arach))

insect_index = df[df["class"]=="Insecta"].index
insect = []

for index in insect_index:
  insect.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Insecta"] = list(set(insect))

acti_index = df[df["class"]=="Actinopterygii"].index
acti = []

for index in acti_index:
  acti.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Actinopterygii"] = list(set(acti))

aves_index = df[df["class"]=="Aves"].index
aves = []

for index in aves_index:
  aves.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Aves"] = list(set(aves))

mamma_index = df[df["class"]=="Mammalia"].index
mamma = []

for index in mamma_index:
  mamma.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Mammalia"] = list(set(mamma))

repti_index = df[df["class"]=="Reptilia"].index
repti = []

for index in repti_index:
  repti.append(np.argmax(dataset[index]["leaf_label"]))

label_dict["Reptilia"] = list(set(repti))

label_dict.keys()

dict_keys(['Arachnida', 'Insecta', 'Actinopterygii', 'Aves', 'Mammalia', 'Reptilia'])

In [5]:
pos_dict = {
  "Arachnida": 0,
  "Insecta": 1,
  "Actinopterygii": 2,
  "Amphibia": 3,
  "Aves": 4,
  "Mammalia": 5,
  "Reptilia": 6
}

confusion_matrix = {
  "Arachnida": [0, 0, 0, 0, 0, 0, 0],
  "Insecta": [0, 0, 0, 0, 0, 0, 0],
  "Actinopterygii": [0, 0, 0, 0, 0, 0, 0],
  "Amphibia": [0, 0, 0, 0, 0, 0, 0],
  "Aves": [0, 0, 0, 0, 0, 0, 0],
  "Mammalia": [0, 0, 0, 0, 0, 0, 0],
  "Reptilia": [0, 0, 0, 0, 0, 0, 0]
}

In [6]:
for x in tqdm(H.test_loader):
  for label, predicted in H.get_label(x):
    for key, array in label_dict.items():
      if label in array:
        true_key = key
      if predicted in array:
        predicted_class = pos_dict[key]
    confusion_matrix[true_key][predicted_class] += 1

  0%|          | 0/405 [00:00<?, ?it/s]

In [7]:
confusion_matrix

{'Arachnida': [0, 0, 0, 0, 0, 0, 0],
 'Insecta': [0, 18, 0, 0, 15, 0, 0],
 'Actinopterygii': [0, 0, 0, 0, 0, 0, 0],
 'Amphibia': [0, 0, 0, 0, 0, 0, 0],
 'Aves': [0, 1, 0, 0, 2761, 0, 0],
 'Mammalia': [0, 1, 0, 0, 433, 0, 0],
 'Reptilia': [0, 0, 0, 0, 4, 0, 0]}